In [1]:
import pandas as pd
import numpy as np
import folium
from haversine import haversine, Unit
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

import warnings
warnings.filterwarnings('ignore')

In [2]:
data1 = pd.read_excel('data/al_Mega_address.xlsx')
del data1['Unnamed: 0']
data1.head(1)

,ID,18/1 거래금액(만원),18/1~18/2,증감률,18/1 cluster,18/2 거래금액(만원),18/2~18/3,증감률.1,18/2 cluster,18/3 거래금액(만원),...,22/1 cluster,18/1 거래금액(만원).1,22/2 cluster,전용면적,건축년도,인접역 수,Brand,BrandPower,위도,경도
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,218666.7,3666.7,1.676844,1,215000.0,0.0,0.0,2,215000.0,...,0,129125.0,1,80,1987.0,0,0,0,37.483372,127.053437


In [3]:
data2 = pd.read_excel('data/NewRent.xlsx')
del data2['Unnamed: 0']
data2.head(1)

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,보증금(만원),월세(만원),층,건축년도,도로명,분기
0,서울특별시 강남구 개포동,SH대치1단지아파트,월세,30,2018-03-01,"3,865",17,11,1991.0,개포로109길 5,2018/1


In [4]:
data2['ID'] = data2['시군구'] + ' ' + data2['단지명'] + ' ' + data2['전용면적(㎡)'].astype('str')
data2.head(1)

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,보증금(만원),월세(만원),층,건축년도,도로명,분기,ID
0,서울특별시 강남구 개포동,SH대치1단지아파트,월세,30,2018-03-01,"3,865",17,11,1991.0,개포로109길 5,2018/1,서울특별시 강남구 개포동 SH대치1단지아파트 30


In [5]:
data2['계약년월'] = pd.to_datetime(data2['계약년월'], format='%Y%m')
data2['분기'], data2['년도'] = (data2['계약년월'].dt.quarter, data2['계약년월'].dt.year)
data2['분기'] = data2[['년도','분기']].apply(lambda row:'/'.join(row.values.astype(str)), axis=1)
data2.head(5)

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,보증금(만원),월세(만원),층,건축년도,도로명,분기,ID,년도
0,서울특별시 강남구 개포동,SH대치1단지아파트,월세,30,2018-03-01,"3,865",17,11,1991.0,개포로109길 5,2018/1,서울특별시 강남구 개포동 SH대치1단지아파트 30,2018
1,서울특별시 강남구 개포동,개포2차현대아파트,월세,80,2018-02-01,"39,000",30,4,1988.0,언주로 103,2018/1,서울특별시 강남구 개포동 개포2차현대아파트 80,2018
2,서울특별시 강남구 개포동,개포2차현대아파트,전세,80,2018-04-01,"52,000",0,9,1988.0,언주로 103,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80,2018
3,서울특별시 강남구 개포동,개포2차현대아파트,전세,80,2018-05-01,"50,000",0,5,1988.0,언주로 103,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80,2018
4,서울특별시 강남구 개포동,개포2차현대아파트,전세,80,2018-05-01,"56,000",0,9,1988.0,언주로 103,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80,2018


In [6]:
data2.drop(data2[data2['전월세구분'] == '월세'].index, inplace=True)
data2.head()

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,보증금(만원),월세(만원),층,건축년도,도로명,분기,ID,년도
2,서울특별시 강남구 개포동,개포2차현대아파트,전세,80,2018-04-01,"52,000",0,9,1988.0,언주로 103,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80,2018
3,서울특별시 강남구 개포동,개포2차현대아파트,전세,80,2018-05-01,"50,000",0,5,1988.0,언주로 103,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80,2018
4,서울특별시 강남구 개포동,개포2차현대아파트,전세,80,2018-05-01,"56,000",0,9,1988.0,언주로 103,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80,2018
5,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,전세,70,2018-01-01,"38,000",0,3,1987.0,언주로 3,2018/1,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 70,2018
6,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,전세,80,2018-01-01,"43,000",0,1,1987.0,언주로 3,2018/1,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,2018


In [7]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 579046 entries, 2 to 864924
Data columns (total 13 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   시군구      579046 non-null  object        
 1   단지명      578129 non-null  object        
 2   전월세구분    579046 non-null  object        
 3   전용면적(㎡)  579046 non-null  int64         
 4   계약년월     579046 non-null  datetime64[ns]
 5   보증금(만원)  579046 non-null  object        
 6   월세(만원)   579046 non-null  object        
 7   층        579046 non-null  int64         
 8   건축년도     578907 non-null  float64       
 9   도로명      579046 non-null  object        
 10  분기       579046 non-null  object        
 11  ID       578129 non-null  object        
 12  년도       579046 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(8)
memory usage: 61.8+ MB


In [8]:
data2['보증금(만원)'] = data2['보증금(만원)'].str.replace(' ', '')
data2['보증금(만원)'] = data2['보증금(만원)'].str.replace(',', '')
data2['보증금(만원)'].astype(int)

2         52000
3         50000
4         56000
5         38000
6         43000
          ...  
864918    45000
864919    32500
864920    35000
864923    29400
864924    23000
Name: 보증금(만원), Length: 579046, dtype: int32

In [10]:
table = pd.pivot_table(data2, values=['보증금(만원)'], index=['분기'], columns=['ID'], aggfunc=np.mean)
table.round(1)

보증금(만원)                                  \
ID     서울특별시 강남구 개포동 개포2차현대아파트 80 서울특별시 강남구 개포동 개포6차우성아파트1동~8동 50   
분기                                                                  
2018/1                        NaN                             NaN   
2018/2               1.733350e+14                    3.900000e+04   
2018/3               2.650025e+09                    4.000000e+04   
2018/4                        NaN                    3.100000e+04   
2019/1               5.000000e+04                    2.600000e+04   
2019/2               5.000000e+04                    3.000000e+04   
2019/3               2.000017e+14                    8.333417e+13   
2019/4               2.750032e+09                             NaN   
2020/1               5.300000e+04                             NaN   
2020/2               2.050022e+14                    1.650015e+09   
2020/3               2.000018e+14                             NaN   
2020/4               3.250038e+09                             NaN   
2021/1               2.666691e+14                             NaN   
2021/2               2.625026e+09                    1.333345e+14   
2021/3               3.000025e+09                    5.000000e+04   
2021/4               5.040000e+04                             NaN   
2022/1               3.200032e+09                             NaN   
2022/2               2.700035e+09                    1.750018e+09   

                                                                        \
ID     서울특별시 강남구 개포동 개포6차우성아파트1동~8동 70 서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80   
분기                                                                       
2018/1                    1.900020e+09                    4.777821e+43   
2018/2                    2.000019e+09                    9.000100e+23   
2018/3                             NaN                    4.777822e+43   
2018/4                    1.000009e+19                    5.125050e+38   
2019/1                             NaN                    5.125048e+38   
2019/2                    7.500078e+18                    5.833395e+28   
2019/3                    3.000000e+04                    8.620084e+23   
2019/4                    1.266679e+14                    1.075010e+19   
2020/1                             NaN                    7.166742e+28   
2020/2                    1.250012e+14                    7.200072e+23   
2020/3                    2.000018e+09                    5.375054e+38   
2020/4                    1.750019e+09                    1.100011e+19   
2021/1                    2.250025e+09                    8.800100e+23   
2021/2                    3.000000e+04                    8.666762e+28   
2021/3                    1.706267e+09                    1.125011e+19   
2021/4                    7.875100e+18                    2.583359e+14   
2022/1                    3.950000e+04                    8.625056e+38   
2022/2                    3.000028e+09                    1.250015e+19   

                                                              \
ID     서울특별시 강남구 개포동 개포더샵트리에 110 서울특별시 강남구 개포동 개포래미안포레스트 100   
분기                                                             
2018/1                       NaN                         NaN   
2018/2                       NaN                         NaN   
2018/3                       NaN                         NaN   
2018/4              1.000000e+05                         NaN   
2019/1                       NaN                         NaN   
2019/2                       NaN                         NaN   
2019/3                       NaN                         NaN   
2019/4                       NaN                         NaN   
2020/1                       NaN                         NaN   
2020/2                       NaN                6.500008e+10   
2020/3                       NaN                1.650002e+58   
2020/4                       NaN               8.888898e+105   
2021/1                       NaN                3.600004e+28 